In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import pylab as plt

# Project

### Description

This is a learning project. The point of it is to run analysis on the HoI4 air combat algorithm. It will use python and MatPlotLib to make the calculations and display them as graphs to make analysis and comparison easier.

### Preparation

We need to set up the basic data that we will be running the algorithm over to see the results. We will be using objects to store and pass data. 

In [ ]:
Class Plane:
    def _init_(self, attack, defence, agility, speed, cost)
        self.attack = attack
        self.defence = defence
        self.agility = agility
        self.speed = speed
        self.cost = cost
